# Create Prediction Tiles

**Timm Nawrocki**  
Alaska Center for Conservation Science  
2019-04-05

In [1]:
# -*- coding: utf-8 -*-
# ---------------------------------------------------------------------------
# Create Prediction Tiles
# Author: Timm Nawrocki
# Created on: 2019-04-05
# Usage: Must be executed as a Jupyter Notebook in an ArcGIS Pro Python 3 installation.
# Description: "Create Prediction Tiles" generates the sampling routes along the Nuyakuk and Nushagak rivers and along the Ahklun Lakes and Wood River, buffers the sampling routes by 1 mile to create sampling areas, selects subwatersheds (6th level hydrologic units) along the sampling routes, and exports prediction tile rasters for each subwatershed.
# ---------------------------------------------------------------------------

## 1. Initialize Environment

In [2]:
# Import packages
import arcpy
from arcpy.sa import *
import datetime
import os
import time

# Set overwrite option
arcpy.env.overwriteOutput = True

# Set root directory
drive = 'E:/'
root_directory = os.path.join(drive, 'ACCS_Work/Projects/VegetationEcology/BristolBay_Vegetation/Project_GIS')

# Set arcpy working environment
arcpy.env.workspace = os.path.join(root_directory, 'BristolBay_Vegetation.gdb')

# Define script paths
script_directory = os.path.join(drive, 'ACCS_Work/Repositories/southwest-alaska-moose/modules')
arcpy_geoprocessing_script = os.path.join(script_directory, 'arcpy_geoprocessing.py')

# Define input datasets
nhd_flowlines = os.path.join(root_directory, 'Data_Input/source_data/inland_waters/NHD_H_02_GDB.gdb/Hydrography/NHDFlowline')
subwatersheds = os.path.join(root_directory, 'Data_Input/source_data/inland_waters/NHD_H_02_GDB.gdb/WBD/WBDHU12')
nuyakuk_segments = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_NuyakukNushagakRivers')
wood_segments = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_WoodRiver')
aleknagik_lakes = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_AleknagikLakes')
dillingham_roads = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_DillinghamRoads')
area_of_interest = os.path.join(root_directory, 'Data_Input/area_of_interest/area_of_interest.tif')

# Define output feature class
nuyakuk_route = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_NuyakukNushagak_Route')
aleknagik_route = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_AleknagikWood_Route')
nuyakuk_area = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_NuyakukNushagak_Area')
aleknagik_area = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_AleknagikWood_Area')
nuyakuk_subwatersheds = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_NuyakukNushagak_Subwatersheds')
aleknagik_subwatersheds = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_AleknagikWood_Subwatersheds')

# Define output folder for watershed tiles
nuyakuk_rasters = os.path.join(root_directory, 'Data_Output/prediction_rasters/nuyakuk')
aleknagik_rasters = os.path.join(root_directory, 'Data_Output/prediction_rasters/aleknagik')
nuyakuk_grids = os.path.join(root_directory, 'Data_Output/prediction_grids/nuyakuk')
aleknagik_grids = os.path.join(root_directory, 'Data_Output/prediction_grids/aleknagik')

In [3]:
# Import and execute arcpy_geoprocessing.py
try:
    exec(open(arcpy_geoprocessing_script).read())
except:
    print("Error loading arcpy_geoprocessing.py; ensure that script directory is correct:")
    print(script_directory)
    quit()

## 2. Generate sampling routes

In [ ]:
# Define a function to generate sampling route
def sampling_route(**kwargs):
    """
    Description: generates sampling routes from a single or multiple feature classes
    Inputs: 'input_array' -- an array containing one or more line feature classes
            'output_array' -- the output should be an array of a single output line feature class
    """
    # Start timing function execution
    start = time.time()
    # Parse key word argument inputs
    input_array = kwargs['input_array']
    output_feature = kwargs['output_array'][0]
    print('Generating sampling route...')
    # Define intermediate datasets
    initial_route = os.path.join(arcpy.env.workspace, 'route_initial')
    # If input array includes more than one feature class then merge line feature classes into one and dissolve
    if len(input_array) > 1:
        arcpy.Merge_management(input_array, initial_route)
        arcpy.Dissolve_management(initial_route, output_feature, '', '', 'MULTI_PART', 'DISSOLVE_LINES')
    # If input array contains one feature class then dissolve
    elif len(input_array) == 1:
        input_feature = input_array[0]
        arcpy.Dissolve_management(input_feature, output_feature, '', '', 'MULTI_PART', 'DISSOLVE_LINES')
    # Delete intermediate dataset
    if arcpy.Exists(initial_route):
        arcpy.Delete_management(initial_route)
    # End timing function execution and calculate elapsed time
    end = time.time()
    elapsed = int(end - start)
    success_time = datetime.datetime.now()
    # Report process success
    print('Successfully created sampling route...')
    print('----------')
    out_process = 'Succeeded at {0} (Elapsed time: {1})'.format(success_time.strftime("%Y-%m-%d %H:%M"),
                                                                datetime.timedelta(seconds=elapsed))
    return out_process

In [ ]:
# Define key word values for Nuyakuk and Nushagak Rivers
nuyakuk_route_inputs = [nuyakuk_segments]
nuyakuk_route_outputs = [nuyakuk_route]

# Create key word arguments
nuyakuk_route_kwargs = {'input_array' : nuyakuk_route_inputs,
                        'output_array' : nuyakuk_route_outputs
                       }

# Create the Nuyakuk and Nushagak rivers sampling route
arcpy_geoprocessing(sampling_route, **nuyakuk_route_kwargs)

In [ ]:
# Define key word values for Aleknagik lakes and Wood River
aleknagik_route_inputs = [aleknagik_lakes, wood_segments, dillingham_roads]
aleknagik_route_outputs = [aleknagik_route]

# Create key word arguments
aleknagik_route_kwargs = {'input_array' : aleknagik_route_inputs,
                          'output_array' : aleknagik_route_outputs
                         }

# Create the Alegnagik and Wood River sampling route
arcpy_geoprocessing(sampling_route, **aleknagik_route_kwargs)

## 3. Create sampling areas

In [ ]:
# Define a function to generate sampling area and subwatersheds from sampling route
def sampling_area(**kwargs):
    """
    Description: generates sampling area by buffering sampling route by one mile and selects subwatersheds
    Inputs: 'input_array' -- an array containing a sampling route line feature class, the subwatersheds feature class, and an optional lakes line feature class
            'output_array' -- an array containing a sampling area polygon feature class and a selected subwatersheds feature class
    """
     # Start timing function execution
    start = time.time()
    # Parse key word argument inputs
    input_array = kwargs['input_array']
    input_feature = input_array[0]
    subwatersheds = input_array[1]
    output_area = kwargs['output_array'][0]
    output_subwatersheds = kwargs['output_array'][1]
    # Define intermediate datasets
    area_buffer = os.path.join(arcpy.env.workspace, 'area_buffer')
    lakes_polygon = os.path.join(arcpy.env.workspace, 'lakes_polygon')
    subwatersheds_NAD83 = os.path.join(arcpy.env.workspace, 'subwatersheds_NAD83')
    # Convert sampling route to sampling area by applying a one mile buffer
    print('Generating one mile buffer of sampling area...')
    if len(input_array) == 2:
        arcpy.Buffer_analysis(input_feature, output_area, '1609 Meters', 'FULL', 'ROUND', 'ALL', '', 'PLANAR')
    elif len(input_array) == 3:
        arcpy.Buffer_analysis(input_feature, area_buffer, '1609 Meters', 'FULL', 'ROUND', 'ALL', '', 'PLANAR')\
        # Remove lakes from buffer area
        lakes_line = input_array[2]
        arcpy.FeatureToPolygon_management(lakes_line, lakes_polygon)
        arcpy.Erase_analysis(area_buffer, lakes_polygon, output_area)
    else:
        print('Invalid input array. Must contain a sampling route line feature class, a subwatersheds feature class, and an optional lakes line feature class.')
        quit()
    # Select subwatersheds that intersect the sampling area and export as new feature class
    print('Selecting subwatersheds that intersect sampling area...')
    subwatersheds_select = arcpy.SelectLayerByLocation_management(subwatersheds, 'INTERSECT', output_area, '', 'NEW_SELECTION', 'NOT_INVERT')
    arcpy.CopyFeatures_management(subwatersheds_select, subwatersheds_NAD83)
    spatial_reference = arcpy.SpatialReference(3338)
    arcpy.Project_management(subwatersheds_NAD83, output_subwatersheds, spatial_reference)
    arcpy.AddField_management(output_subwatersheds, 'VALUE', 'SHORT', '', '', '', '', 'NULLABLE', 'NON_REQUIRED', '')
    arcpy.CalculateField_management(output_subwatersheds, 'VALUE', '1', 'PYTHON3', '')
    # Delete intermediate dataset
    if arcpy.Exists(area_buffer):
        arcpy.Delete_management(area_buffer)
    if arcpy.Exists(lakes_polygon):
        arcpy.Delete_management(lakes_polygon)
    if arcpy.Exists(subwatersheds_NAD83):
        arcpy.Delete_management(subwatersheds_NAD83)
    # End timing function execution and calculate elapsed time
    end = time.time()
    elapsed = int(end - start)
    success_time = datetime.datetime.now()
    # Report process success
    print('Successfully created sampling area and subwatersheds...')
    print('----------')
    out_process = 'Succeeded at {0} (Elapsed time: {1})'.format(success_time.strftime("%Y-%m-%d %H:%M"),
                                                                datetime.timedelta(seconds=elapsed))
    return out_process

In [ ]:
# Define key word values for Nuyakuk and Nushagak Rivers
nuyakuk_area_inputs = [nuyakuk_route, subwatersheds]
nuyakuk_area_outputs = [nuyakuk_area, nuyakuk_subwatersheds]

# Create key word arguments
nuyakuk_area_kwargs = {'input_array' : nuyakuk_area_inputs,
                       'output_array' : nuyakuk_area_outputs
                      }

# Create the Nuyakuk and Nushagak rivers sampling area and subwatersheds
arcpy_geoprocessing(sampling_area, **nuyakuk_area_kwargs)

In [ ]:
# Define key word values for Aleknagik lakes and Wood River
aleknagik_area_inputs = [aleknagik_route, subwatersheds, aleknagik_lakes]
aleknagik_area_outputs = [aleknagik_area, aleknagik_subwatersheds]

# Create key word arguments
aleknagik_area_kwargs = {'input_array' : aleknagik_area_inputs,
                         'output_array' : aleknagik_area_outputs
                        }

# Create the Alegnagik and Wood River sampling area and subwatersheds
arcpy_geoprocessing(sampling_area, **aleknagik_area_kwargs)

## 4. Create subwatershed tiles

In [4]:
# Define a function to generate subwatershed prediction tiles from 
def subwatershed_tiles(**kwargs):
    """
    Description: generates subwatershed tiles using the area of interest to define cell size and grid
    Inputs: 'input_array' -- an array containing a the subwatersheds feature class and an area of interest
            'raster_path' -- the output location for tile rasters
            'grid_path' -- the output location for point grid shapefiles
    """
     # Start timing function execution
    start = time.time()
    # Parse key word argument inputs
    subwatersheds = kwargs['input_array'][0]
    area_of_interest = kwargs['input_array'][1]
    raster_path = kwargs['raster_path']
    grid_path = kwargs['grid_path']
    # Define intermediate datasets
    subwatershed_unit = os.path.join(arcpy.env.workspace, 'subwatershed_unit')
    # Loop through each feature in subwatersheds feature class and convert to raster
    with arcpy.da.SearchCursor(subwatersheds, ['HUC12']) as cursor:
        for row in cursor:
            HUC12 = row[0]
            # Define output datasets
            output_raster = os.path.join(raster_path, HUC12 + '.tif')
            output_grid = os.path.join(grid_path, HUC12 + '.shp')
            # Process HUC only if output grid does not already exist
            if arcpy.Exists(output_grid) == False:
                print('----------')
                print('Converting hydrologic unit {0}...'.format(str(HUC12)))
                where_clause = "HUC12 = '{0}'".format(str(HUC12))
                subwatershed_select = arcpy.SelectLayerByAttribute_management(subwatersheds, 'NEW_SELECTION', where_clause, 'NON_INVERT')
                arcpy.CopyFeatures_management(subwatershed_select, subwatershed_unit)
                # Set the snap raster and cell size environments
                arcpy.env.snapRaster = area_of_interest
                arcpy.env.cellSize = area_of_interest
                # Define cell size
                print('Converting feature to raster...')
                cell_size = arcpy.GetRasterProperties_management(area_of_interest, "CELLSIZEX")
                # Convert from polygon to raster and raster to point to create point grid
                arcpy.PolygonToRaster_conversion(subwatershed_unit, "VALUE", output_raster, "CELL_CENTER", "NONE", cell_size)
                # Determine if raster contains only NoData values
                noData = int(arcpy.GetRasterProperties_management(output_raster, 'ALLNODATA')[0])
                if noData == 0:
                # Convert raster to point grid
                    print('Converting raster to point grid...')
                    arcpy.RasterToPoint_conversion(output_raster, output_grid, "VALUE")
                    # Add XY Coordinates to feature class in the NAD_1983_Alaska_Albers projection
                    arcpy.AddXY_management(output_grid)
                elif noData == 1:
                    print('All values for this watershed are nodata...')
                    arcpy.Delete_management(output_raster)
    # End timing function execution and calculate elapsed time
    end = time.time()
    elapsed = int(end - start)
    success_time = datetime.datetime.now()
    # Report process success
    print('Successfully created subwatershed raster tiles and point grids...')
    print('----------')
    out_process = 'Succeeded at {0} (Elapsed time: {1})'.format(success_time.strftime("%Y-%m-%d %H:%M"),
                                                                datetime.timedelta(seconds=elapsed))
    return out_process

In [5]:
# Define key word values for Nuyakuk and Nushagak Rivers
nuyakuk_tiles_inputs = [nuyakuk_subwatersheds, area_of_interest]

# Create key word arguments
nuyakuk_tiles_kwargs = {'input_array' : nuyakuk_tiles_inputs,
                        'raster_path' : nuyakuk_rasters,
                        'grid_path' : nuyakuk_grids
                       }

# Create the Nuyakuk and Nushagak rivers subwatershed raster tiles and point grids
arcpy_geoprocessing(subwatershed_tiles, check_output = False, **nuyakuk_tiles_kwargs)

----------
Converting hydrologic unit 190303011706...
Converting feature to raster...
Converting raster to point grid...
----------
Converting hydrologic unit 190303011010...
Converting feature to raster...
Converting raster to point grid...
----------
Converting hydrologic unit 190303030205...
Converting feature to raster...
Converting raster to point grid...
----------
Converting hydrologic unit 190303030204...
Converting feature to raster...
Converting raster to point grid...
----------
Converting hydrologic unit 190303012008...
Converting feature to raster...
Converting raster to point grid...
----------
Converting hydrologic unit 190303011903...
Converting feature to raster...
Converting raster to point grid...
----------
Converting hydrologic unit 190303011703...
Converting feature to raster...
Converting raster to point grid...
----------
Converting hydrologic unit 190303011705...
Converting feature to raster...
Converting raster to point grid...
----------
Converting hydrologic

In [6]:
# Define key word values for Aleknagik lakes and Wood River
aleknagik_tiles_inputs = [aleknagik_subwatersheds, area_of_interest]

# Create key word arguments
aleknagik_tiles_kwargs = {'input_array' : aleknagik_tiles_inputs,
                          'raster_path' : aleknagik_rasters,
                          'grid_path' : aleknagik_grids
                         }

# Create the Aleknagik and Wood River subwatershed raster tiles and point grids
arcpy_geoprocessing(subwatershed_tiles, check_output = False, **aleknagik_tiles_kwargs)

----------
Converting hydrologic unit 190303060203...
Converting feature to raster...
Converting raster to point grid...
----------
Converting hydrologic unit 190303060205...
Converting feature to raster...
Converting raster to point grid...
----------
Converting hydrologic unit 190303060206...
Converting feature to raster...
Converting raster to point grid...
----------
Converting hydrologic unit 190303060201...
Converting feature to raster...
Converting raster to point grid...
----------
Converting hydrologic unit 190303040503...
Converting feature to raster...
Converting raster to point grid...
----------
Converting hydrologic unit 190303060207...
Converting feature to raster...
Converting raster to point grid...
----------
Converting hydrologic unit 190303040505...
Converting feature to raster...
Converting raster to point grid...
----------
Converting hydrologic unit 190303031108...
Converting feature to raster...
Converting raster to point grid...
----------
Converting hydrologic